In [1]:
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from collections import defaultdict # Dictionaries with default values
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import numpy as np

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sddjl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
## importing data sets and dropping nan's
df = pd.read_csv('GFM_data.csv',sep = '\t')
df = df.loc[df['Text'].dropna().index]
df.head()

,Unnamed: 0,Url,Category,Position,Title,Location,Amount_Raised,Goal,Number_of_Donations,Length_of_Fundraising,FB_Shares,Number_of_Donors,Followers,Text
0,0,https://www.gofundme.com/f/justiceforjacobblake,Medical,0,Justice for Jacob Blake,"Kenosha, WI",2297930.0,3000000.0,73K,93 days 12:02:38.405126000,118K,72.5K,73.4K,On August 23rd my son was shot multiple times ...
1,0,https://www.gofundme.com/f/official-navajo-nat...,Medical,0,Official Navajo Nation COVID-19 Relief Fund,"Window Rock, AZ",1862040.0,1000000.0,22.5K,205 days 12:02:39.366241000,71.7K,21.9K,22K,\r\nThe Navajo Nation COVID-19 Fund has been e...
2,0,https://www.gofundme.com/f/help-a-front-line-n...,Medical,0,Help a front line nurse and baby get proper care,"Randolph, NJ",954793.0,1200000.0,19K,215 days 12:02:40.340314000,16.4K,18.3K,17.9K,"On Sunday, April 12, Sylvia Leroy, a pregnant ..."
3,0,https://www.gofundme.com/f/Tommy-Rivers-Rest-Up,Medical,1,"Rest up, Tommy, we'll see you soon","Scottsdale, AZ",673179.0,1000000.0,11.3K,131 days 12:02:41.464483000,21.3K,10.3K,10.4K,"First, thank you for being here. Tommy Rivers ..."
4,0,https://www.gofundme.com/f/brandon039s-medical...,Medical,1,OFFICIAL BRANDON SAENZ MEDICAL FUND,"Tyler, TX",570529.0,750000.0,24.7K,175 days 12:02:42.383091000,5.5K,24.3K,24.5K,My name is Melissa Green and I am the mother o...


## Text preprocessing using Regex

In [3]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
REPLACE_IP_ADDRESS = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')

def clean_text(x):
    ## normalizing text by stripping white space and lower casing
    x =  x.lower().strip()
    ## removing urls
    x = re.sub(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))', '', x)
    ## removing phone numbers
    x = re.sub('\([0-9]{3}\)\s*[0-9]{3}-[0-9]{4}','',x)
    ## strip all non alphanumeric things
    x = re.sub('\n',' ',x)
    x = re.sub("[^a-zA-Z0-9 #]",'',x)
    x = re.sub("\s+",' ',x)
    text = x.replace('\n', ' ').lower()# lowercase text
    text = REPLACE_IP_ADDRESS.sub('', text) # remove ip address
    text = REPLACE_BY_SPACE_RE.sub(' ',text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('',text)# delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join([w for w in text.split() if not w in STOPWORDS])# delete stopwords from text
    
    return text

In [4]:
df['Text'] = df['Text'].apply(clean_text)

## Splitting into train and test set

In [5]:
categories = {i: idx for idx,i in enumerate(df['Category'].unique())}

In [6]:
from sklearn.model_selection import train_test_split
X = df['Text']
y = [categories[i] for i in df['Category']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## BoW (Bag of Words)


In [7]:
from scipy import sparse as sp_sparse

In [8]:
# Dictionary of all words from train corpus with their counts.
words_counts = {}
for comments in X_train:
    for word in comments.split():
        if word not in words_counts:
            words_counts[word] = 0
        words_counts[word] += 1
##most pop words        
DICT_SIZE = 10000
POPULAR_WORDS = sorted(words_counts, key=words_counts.get, reverse=True)[:DICT_SIZE]
## same dics but flopped
WORDS_TO_INDEX = {key: rank for rank, key in enumerate(POPULAR_WORDS, 0)}
INDEX_TO_WORDS = {index:word for word, index in WORDS_TO_INDEX.items()}
ALL_WORDS = WORDS_TO_INDEX.keys()

In [9]:
def my_bag_of_words(text, words_to_index, dict_size):
    result_vector = np.zeros(dict_size)
    for word in text.split(' '):
        if word in words_to_index:
            result_vector[words_to_index[word]] +=1
    return result_vector

X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape, '\nX_val shape ', X_test_mybag.shape)

X_train shape  (646, 10000) 
X_val shape  (216, 10000)


In [10]:
print(X_train_mybag[0])


  (0, 0)	1.0
  (0, 1)	5.0
  (0, 2)	5.0
  (0, 3)	5.0
  (0, 4)	2.0
  (0, 5)	2.0
  (0, 6)	3.0
  (0, 7)	5.0
  (0, 8)	1.0
  (0, 9)	2.0
  (0, 10)	1.0
  (0, 13)	1.0
  (0, 14)	3.0
  (0, 15)	3.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 21)	1.0
  (0, 22)	2.0
  (0, 25)	2.0
  (0, 26)	1.0
  (0, 32)	1.0
  (0, 35)	1.0
  (0, 40)	4.0
  (0, 42)	1.0
  (0, 48)	2.0
  :	:
  (0, 8605)	1.0
  (0, 8606)	1.0
  (0, 8607)	1.0
  (0, 8608)	1.0
  (0, 8609)	1.0
  (0, 8610)	1.0
  (0, 8611)	1.0
  (0, 8612)	1.0
  (0, 8613)	1.0
  (0, 8614)	1.0
  (0, 8615)	1.0
  (0, 8616)	1.0
  (0, 8617)	1.0
  (0, 8618)	1.0
  (0, 8619)	1.0
  (0, 8620)	1.0
  (0, 8621)	1.0
  (0, 8622)	1.0
  (0, 8623)	1.0
  (0, 8624)	1.0
  (0, 8625)	1.0
  (0, 8626)	1.0
  (0, 8627)	1.0
  (0, 8628)	1.0
  (0, 8629)	1.0


In [11]:
POPULAR_WORDS[:10]

['help',
 'family',
 'us',
 'support',
 'community',
 'time',
 'many',
 'years',
 'life',
 'would']

## TF-IDF¶


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
## creating tfidf vector
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5)
## transforming it
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


In [14]:
print(X_train_tfidf[0])

  (0, 3544)	0.05412889865323257
  (0, 1444)	0.041825336076166786
  (0, 3400)	0.05789711357437934
  (0, 2214)	0.0438671889721343
  (0, 3067)	0.059077065510944285
  (0, 3909)	0.03746364799126061
  (0, 3647)	0.06189158235803169
  (0, 3914)	0.0603961395506785
  (0, 1408)	0.0603961395506785
  (0, 3264)	0.06189158235803169
  (0, 2074)	0.06361794438321094
  (0, 85)	0.056829717211213264
  (0, 3884)	0.05013442986958022
  (0, 1277)	0.12378316471606338
  (0, 1292)	0.0603961395506785
  (0, 1991)	0.04809257697361271
  (0, 1651)	0.04452615463414748
  (0, 1938)	0.03922015589005254
  (0, 3350)	0.06189158235803169
  (0, 1442)	0.024804525859215356
  (0, 1505)	0.038405310400233705
  (0, 168)	0.038405310400233705
  (0, 1588)	0.046773502933878504
  (0, 930)	0.047635403893281074
  (0, 2486)	0.043244851912797075
  :	:
  (0, 2605)	0.0728582841274634
  (0, 516)	0.03965094689626415
  (0, 1637)	0.032451154791935395
  (0, 3883)	0.03764575257733272
  (0, 106)	0.06361794438321094
  (0, 2578)	0.26099120042730084
  (

## Classification

In [15]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
def train_classifier(X_train, y_train, C, regularisation):
    model = OneVsRestClassifier(LogisticRegression(penalty=regularisation, C=C, max_iter=10000)).fit(X_train, y_train)
    return model

In [16]:
classifier_mybag = train_classifier(X_train_mybag, y_train, C = 4, regularisation = 'l2')
classifier_tfidf = train_classifier(X_train_tfidf, y_train, C = 4, regularisation = 'l2')

C:\Users\sddjl\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [17]:
y_test_predicted_labels_mybag = classifier_mybag.predict(X_test_mybag)
y_test_predicted_labels_tfidf = classifier_tfidf.predict(X_test_tfidf)

In [18]:
y_test_predicted_scores_mybag = classifier_mybag.decision_function(X_test_mybag)
y_test_predicted_scores_tfidf = classifier_tfidf.decision_function(X_test_tfidf)

In [19]:
y_test_predicted_scores_mybag

array([[ -4.38645854,  -5.1065999 ,  -5.14648542, ...,   1.00852141,
         -5.15789754,  -3.87148621],
       [ -6.4118101 ,  -3.4613966 ,  -2.69392407, ...,  -9.86056155,
         -7.62976117,  -8.08212573],
       [ -5.70047232,  -4.55874478,  -4.25212291, ...,  -7.75770776,
         -5.86048083,  -3.67080258],
       ...,
       [ -4.50198581,  -4.71379864,  -5.13409493, ...,  -2.4644677 ,
         -3.20029613,  -2.6684269 ],
       [ -6.14127964,  -7.87671075,  -1.25931473, ..., -12.00931522,
         -8.3103033 ,  -8.22532301],
       [-10.44384173, -13.10562152, -10.15969348, ...,  -9.30484935,
         -7.02950308,  -9.32500507]])

## Evaluation


In [20]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [21]:
classifier_tfidf = train_classifier(X_train_tfidf, y_train, C = 50, regularisation = 'l2')

y_test_predicted_labels_mybag = classifier_mybag.predict(X_test_mybag)
y_test_predicted_labels_tfidf = classifier_tfidf.predict(X_test_tfidf)

def print_evaluation_scores(y_test, predicted):
    
    print('Accuracy: ', accuracy_score(y_test, predicted, normalize=False))
    print('F1-score weighted: ', f1_score(y_test, predicted, average='weighted'))
    
print('Bag-of-words\n')
print_evaluation_scores(y_test, y_test_predicted_labels_mybag)
print('\nTfidf\n')
print_evaluation_scores(y_test, y_test_predicted_labels_tfidf)

Bag-of-words

Accuracy:  114
F1-score weighted:  0.5250867663267459

Tfidf

Accuracy:  117
F1-score weighted:  0.5330059430475279


C:\Users\sddjl\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
